<a href="https://colab.research.google.com/github/Pedro0Oliveira/2024.1-TrabalhoN1_Tema2_Questionario_para_otimizacao_da_coleta/blob/main/Segundo_Projeto_Semin%C3%A1rio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import heapq

# Credenciais autorizadas
credenciais_autorizadas = {
    '20241012110010': '2345678'
}

def autenticar_usuario():
    while True:
        matricula = input('Digite sua matrícula: ').strip()
        senha = input('Digite sua senha: ').strip()

        if credenciais_autorizadas.get(matricula) == senha:
            print('Autenticação bem-sucedida!')
            return True
        else:
            print('Matrícula ou senha incorretos. Tente novamente.')

# Função para encontrar a rota mais curta usando o algoritmo de Dijkstra
def dijkstra(mapa_distancias, inicio, destino):
    distancias = {no: float('inf') for no in mapa_distancias}
    distancias[inicio] = 0
    caminho = {no: None for no in mapa_distancias}

    fila_prioridade = [(0, inicio)]

    while fila_prioridade:
        distancia_atual, no_atual = heapq.heappop(fila_prioridade)

        if no_atual == destino:
            break

        for vizinho, peso in mapa_distancias.get(no_atual, {}).items():
            distancia = distancia_atual + peso

            if distancia < distancias[vizinho]:
                distancias[vizinho] = distancia
                caminho[vizinho] = no_atual
                heapq.heappush(fila_prioridade, (distancia, vizinho))

    rota = []
    no = destino
    while no is not None:
        rota.append(no)
        no = caminho[no]
    rota.reverse()

    return rota, distancias[destino]

def obter_valor_valido(mensagem, min_valor, max_valor):
    while True:
        try:
            valor = float(input(mensagem))
            if min_valor <= valor <= max_valor:
                return valor
            else:
                print(f"Erro: Digite um valor entre {min_valor} e {max_valor}.")
        except ValueError:
            print("Erro: Digite um valor numérico válido.")

def calcular_rotas_para_coletar_lixo(mapa_distancias, bairros, lixo_por_bairro, capacidade_caminhao, meta_diaria, inicio, taxa_consumo_combustivel):
    rotas = []
    consumo_combustivel_total = 0
    total_lixo_coletado = 0
    caminhoes_utilizados = 0

    bairros_restantes = list(bairros)
    total_bairros = len(bairros_restantes)

    # Remover o bairro inicial dos bairros restantes para garantir que ele seja o primeiro a ser coletado
    if inicio in bairros_restantes:
        bairros_restantes.remove(inicio)

    while total_lixo_coletado < meta_diaria and bairros_restantes:
        carga_atual = 0
        rota_atual = []
        distancia_total_coleta = 0
        bairro_atual = inicio

        # Primeira coleta obrigatoriamente no bairro de início
        if lixo_por_bairro[bairro_atual] > 0:
            if carga_atual + lixo_por_bairro[bairro_atual] <= capacidade_caminhao:
                carga_atual += lixo_por_bairro[bairro_atual]
                total_lixo_coletado += lixo_por_bairro[bairro_atual]
                lixo_por_bairro[bairro_atual] = 0
            else:
                lixo_restante = capacidade_caminhao - carga_atual
                carga_atual += lixo_restante
                lixo_por_bairro[bairro_atual] -= lixo_restante
                total_lixo_coletado += lixo_restante

            rota_atual.append(bairro_atual)

        # Depois de coletar no bairro de início, seguir o fluxo normal
        while carga_atual < capacidade_caminhao and total_lixo_coletado < meta_diaria and bairros_restantes:
            rota, distancia = dijkstra(mapa_distancias, bairro_atual, 'EstacaoTransbordo')
            if rota:
                distancia_total_coleta += distancia
                bairro_atual = 'EstacaoTransbordo'
                rota_atual.extend(rota[1:])  # Adiciona a rota menos o ponto inicial
            else:
                print("Erro: Rota não encontrada.")
                break

            bairro_atual = bairros_restantes.pop(0)
            if lixo_por_bairro[bairro_atual] > 0:
                if carga_atual + lixo_por_bairro[bairro_atual] <= capacidade_caminhao:
                    carga_atual += lixo_por_bairro[bairro_atual]
                    total_lixo_coletado += lixo_por_bairro[bairro_atual]
                    lixo_por_bairro[bairro_atual] = 0
                else:
                    lixo_restante = capacidade_caminhao - carga_atual
                    carga_atual += lixo_restante
                    lixo_por_bairro[bairro_atual] -= lixo_restante
                    total_lixo_coletado += lixo_restante

                rota_atual.append(bairro_atual)

            if lixo_por_bairro[bairro_atual] <= 0:
                bairro_atual = 'EstacaoTransbordo'

        if rota_atual:
            caminhoes_utilizados += 1
            rota, distancia_total = dijkstra(mapa_distancias, 'EstacaoTransbordo', 'EstacaoTransbordo')
            if rota:
                distancia_total_coleta += distancia_total * 2  # Distância de ida e volta

            consumo_combustivel = (distancia_total_coleta / taxa_consumo_combustivel)
            consumo_combustivel_total += consumo_combustivel

            rotas.append((rota_atual, distancia_total_coleta, consumo_combustivel))

    return rotas, total_lixo_coletado, consumo_combustivel_total, caminhoes_utilizados

def coleta_domiciliar_leste():
    # Dados de produção de lixo por bairro em toneladas
    lixo_por_bairro = {
        'Santos Reis': 6.48,
        'Rocas': 13.87,
        'Ribeira': 3.05,
        'Praia do Meio': 7.16,
        'Cidade Alta': 10.07,
        'Petrópolis': 7.81,
        'Areia Preta': 6.59,
        'Mãe Luiza': 19.15,
        'Alecrim': 35.19,
        'Barro Vermelho': 15.14,
        'Tirol': 22.84,
        'Lagoa Seca': 6.65
    }

    # Distância dos bairros até a estação de transbordo
    mapa_distancias = {
        'Alecrim': {'EstacaoTransbordo': 6.3},
        'Areia Preta': {'EstacaoTransbordo': 11.7},
        'Barro Vermelho': {'EstacaoTransbordo': 8.6},
        'Cidade Alta': {'EstacaoTransbordo': 8.3},
        'Lagoa Seca': {'EstacaoTransbordo': 7.3},
        'Mãe Luiza': {'EstacaoTransbordo': 11.7},
        'Petrópolis': {'EstacaoTransbordo': 11.4},
        'Praia do Meio': {'EstacaoTransbordo': 10.4},
        'Ribeira': {'EstacaoTransbordo': 10},
        'Rocas': {'EstacaoTransbordo': 10},
        'Santos Reis': {'EstacaoTransbordo': 11.5},
        'Tirol': {'EstacaoTransbordo': 9}
    }

    mapa_distancias['EstacaoTransbordo'] = {}

    while True:
        inicio = input('Informe o bairro de partida (ex: Alecrim): ').strip().title()
        if inicio in mapa_distancias:
            break
        else:
            print("Erro: Bairro inválido. Digite um bairro válido.")

    capacidade_caminhao = obter_valor_valido('Informe a capacidade do caminhão em toneladas: ', 9, 30)
    meta_diaria = obter_valor_valido('Informe a meta diária de coleta em toneladas: ', 9, 154)
    taxa_consumo_combustivel = obter_valor_valido('Informe a taxa de consumo de combustível do caminhão (km por litro): ', 1, 20)

    # Calcular o número inteiro de caminhões necessários para a meta diária
    caminhoes_necessarios_meta_diaria = -(-meta_diaria // capacidade_caminhao)  # Arredonda para cima

    # Calcular as rotas para a coleta
    rotas, total_lixo_coletado, consumo_combustivel_total, caminhoes_utilizados = calcular_rotas_para_coletar_lixo(
        mapa_distancias, list(lixo_por_bairro.keys()), lixo_por_bairro, capacidade_caminhao, meta_diaria, inicio, taxa_consumo_combustivel
    )

    # Calcular o número inteiro de caminhões necessários para o lixo total da região
    caminhoes_necessarios_total = -(-154 // capacidade_caminhao)  # Arredonda para cima

    print(f"Número total de caminhões necessários para a meta diária de {meta_diaria:.2f} toneladas: {int(caminhoes_necessarios_meta_diaria)}")
    print(f"Número total de caminhões necessários para o lixo total (154 toneladas): {int(caminhoes_necessarios_total)}")
    print(f"Total de lixo coletado: {total_lixo_coletado:.2f} toneladas")
    print(f"Consumo total de combustível: {consumo_combustivel_total:.2f} litros")

    for i, (rota, distancia, consumo_combustivel) in enumerate(rotas, 1):
        print(f"\nRota do caminhão {i}: {' -> '.join(rota)}")
        print(f"Distância total percorrida: {distancia:.2f} km")
        print(f"Consumo de combustível: {consumo_combustivel:.2f} litros")

if autenticar_usuario():
    coleta_domiciliar_leste()